In [1]:
import pymysql
import yfinance as yf
import import_ipynb

In [2]:
from settings import conn

importing Jupyter notebook from settings.ipynb


In [3]:
conn = conn

In [4]:
cursor = conn.cursor()

In [5]:
def stock_delete(stock_code):
    sql = "DELETE FROM stock where code = '{0}'".format(stock_code)
    cursor.execute(sql)
    conn.commit()

In [ ]:
sql = "SELECT * FROM stock"

In [ ]:
cursor.execute(sql)
stocks = cursor.fetchall()
conn.commit()

In [ ]:
sql = "INSERT INTO record (stock_code, date, volume, price) VALUES (%s, %s, %s, %s)"

In [ ]:
for j in stocks:
    try:
        d = yf.Ticker(j[0]+".KS")
        transaction = d.history("1d")
        transaction = transaction.reset_index()
        transaction = transaction[['Date', 'Close', 'Volume']]
        transaction['Date'] = transaction['Date'].dt.strftime("%Y-%m-%d")
        transaction['Close'] = transaction['Close'].astype('int')
        for k in range(len(transaction)):
            cursor.execute(sql, (j[0], transaction['Date'][k], transaction['Volume'][k], transaction['Close'][k]))
    except:
        stock_delete(j[0]) # yfinance에서 지원하지 않게된(상장 폐지 등) 종목 삭제
conn.commit()

In [ ]:
for i in stocks:
    sql = "SELECT * FROM record WHERE stock_code = {0} ORDER BY date".format(i[0])
    cursor.execute(sql)
    record = cur.fetchall()

    record_day5 = record[-4:]
    start = record_day5[0][1]
    end = record_day5[-1][1]
    record_day5 = [(j[2], j[3]) for j in record_day5]
    avg_volume, avg_price = np.mean(record_day5, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record_day5, axis = 0)
    cursor.execute("INSERT INTO day5 (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))
    conn.commit()

    record_month = record[-29:]
    start = record_month[0][1]
    end = record_month[-1][1]
    record_month = [(j[2], j[3]) for j in record_month]
    avg_volume, avg_price = np.mean(record_month, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record_month, axis = 0)
    cursor.execute("INSERT INTO month (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))
    conn.commit()

    start = record[0][1]
    end = record[-1][1]
    record = [(j[2], j[3]) for j in record]
    avg_volume, avg_price = np.mean(record, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record, axis = 0)
    cursor.execute("INSERT INTO quarter (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))
    conn.commit()

In [ ]:
conn.close()